<a href="https://colab.research.google.com/github/ghazaletlbn/machinelearning/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import sklearn
credits = pd.read_csv('/content/gdrive/My Drive/imdb/credits.csv')
keywords = pd.read_csv('/content/gdrive/My Drive/imdb/keywords.csv')
links_small = pd.read_csv('/content/gdrive/My Drive/imdb/links_small.csv')
movies_metadata = pd.read_csv('/content/gdrive/My Drive/imdb/movies_metadata.csv')
ratings_small = pd.read_csv('/content/gdrive/My Drive/imdb/ratings_small.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
links_small_tmdid = links_small[links_small['tmdbId'].notnull()]['tmdbId']
links_small2 = links_small_tmdid.astype(int)
links_small = links_small[links_small['tmdbId'].isin(links_small_tmdid)]
links_small['tmdbId']=links_small['tmdbId'].astype(int)
links =links_small[['movieId']]
links['id'] = links_small['tmdbId']
links

<ipython-input-3-0f27d5f943e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  links_small['tmdbId']=links_small['tmdbId'].astype(int)
<ipython-input-3-0f27d5f943e2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  links['id'] = links_small['tmdbId']


,movieId,id
0,1,862
1,2,8844
2,3,15602
3,4,31357
4,5,11862
...,...,...
9120,162672,402672
9121,163056,315011
9122,163949,391698
9123,164977,137608


In [ ]:
pd.set_option('display.max_colwidth', None)
movie_data = movies_metadata[['id','title','genres','popularity','vote_average','vote_count']]
movie_data = movie_data[movie_data['id'].str.contains('-') == False]
movie_data['id']=movie_data['id'].astype(int)
otherdata = pd.merge(credits,keywords,on='id')
movie_data = pd.merge(movie_data,otherdata,on='id')
movie_data = movie_data[movie_data['id'].isin(links_small_tmdid)]

In [ ]:
movie_data['vote_count'] = movie_data['vote_count'].replace(0.0, np.nan)
movie_data = movie_data.dropna(axis=0, subset=['vote_count'])
str_data = movie_data[['id','title','genres','cast','crew','keywords']]

In [ ]:
def genres_keywords_cleaner(k):
    s = k.split('}')
    if len(s)==1:
      s=[]
    else:
       p = "'name': '"
       b= len(s)-1;
       for i in range(b):
          s[i] = s[i].partition(p)[2]
          s[i] = s[i][0:( len(s[i]) - 1)]
          s[i] = str.lower(s[i].replace(" ",""))
       s = s[0:b]
    return s;

In [ ]:
def cast_cleaner(k):
    s = k.split('}')
    p1 = "'name': '"
    p2 ="', 'order'"
    if len(s)==1:
       s=[]
    else:
       for i in range(len(s)-1):
         s[i] = s[i].partition(p1)[2]
         s[i] = s[i].partition(p2)[0]
         s[i] = str.lower(s[i].replace(" ",""))
       if(len(s)-1>3):
         s = s[0:4]
       else:
         s =s[0:len(s)-1]
    return s;

In [ ]:
def get_director(k):
    word = "'job': 'Director'"
    p1 = "'name': '"
    p2 ="', 'profile_path'"
    if word in k:
       s = k.split('}')
       for i in range(len(s)):
         if word in s[i]:
             s = s[i]
             break
       s = s.partition(p1)[2]
       s = s.partition(p2)[0]
       s = str.lower(s.replace(" ",""))
       s2 = [s,s]
       return s2;
    else:
       return []

In [ ]:
str_data['crew'] = str_data['crew'].apply(lambda x: get_director(x))
str_data['cast'] = str_data['cast'].apply(lambda x: cast_cleaner(x))
str_data['genres'] = str_data['genres'].apply(lambda x: genres_keywords_cleaner(x))
str_data['keywords'] = str_data['keywords'].apply(lambda x: genres_keywords_cleaner(x))
str_data

<ipython-input-9-f0220af3838f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_data['crew'] = str_data['crew'].apply(lambda x: get_director(x))
<ipython-input-9-f0220af3838f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_data['cast'] = str_data['cast'].apply(lambda x: cast_cleaner(x))
<ipython-input-9-f0220af3838f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles, jimvarney]","[johnlasseter, johnlasseter]","[jealousy, toy, boy, friendship, friends, rivalry, boynextdoor, newtoy, toycomestolife]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst, bradleypierce]","[joejohnston, joejohnston]","[boardgame, disappearance, , newhome, recluse, giantinsect]"
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret, sophialoren]","[howarddeutch, howarddeutch]","[fishing, bestfriend, duringcreditsstinger, oldmen]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine, lelarochon]","[forestwhitaker, forestwhitaker]","[basedonnovel, interracialrelationship, singlemother, divorce, chickflick]"
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort, kimberlywilliams-paisley]","[charlesshyer, charlesshyer]","[baby, midlifecrisis, confidence, aging, daughter, motherdaughterrelationship, pregnancy, contraception, gynecologist]"
...,...,...,...,...,...,...
40952,159550,The Last Brickmaker in America,[drama],"[sidneypoitier, wendycrewson, jayo.sanders, maryalice]","[greggchampion, greggchampion]","[friendship, brickmaking]"
41172,392572,Rustom,"[thriller, romance]","[akshaykumar, , eshagupta, arjanbajwa]","[tinusureshdesai, tinusureshdesai]",[bollywood]
41225,402672,Mohenjo Daro,"[adventure, drama, history, romance]","[hrithikroshan, poojahegde, kabirbedi, arunodaysingh]","[ashutoshgowariker, ashutoshgowariker]",[bollywood]
41391,315011,Shin Godzilla,"[action, adventure, drama, horror, sciencefiction]","[hirokihasegawa, yutakatakenouchi, satomiishihara, kengokora]","[hideakianno, hideakianno]","[monster, godzilla, giantmonster, destruction, kaiju, toyko]"


In [ ]:
keywords_count = str_data.apply(lambda x :pd.Series(x['keywords']),axis=1).stack().reset_index(level=1,drop=True)
keywords_count.name = 'keywords'
keywords_count = keywords_count.value_counts()
keywords_count = keywords_count[keywords_count>1]
keywords_count

<ipython-input-10-cb5bb8ba1b6e>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  keywords_count = str_data.apply(lambda x :pd.Series(x['keywords']),axis=1).stack().reset_index(level=1,drop=True)


independentfilm         610
womandirector           544
murder                  399
duringcreditsstinger    327
basedonnovel            310
                       ... 
lioncub                   2
saw                       2
animalrights              2
policeofficershot         2
panama                    2
Name: keywords, Length: 6654, dtype: int64

In [ ]:
def new_keywords(x):
  w = []
  for i in x:
    if i in keywords_count:
      w.append(i)
  return w

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
str_data['keywords'] = str_data['keywords'].apply(new_keywords)
str_data['keywords'] = str_data['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
str_data

<ipython-input-12-10c1cf6e8620>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_data['keywords'] = str_data['keywords'].apply(new_keywords)
<ipython-input-12-10c1cf6e8620>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_data['keywords'] = str_data['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])


,id,title,genres,cast,crew,keywords
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles, jimvarney]","[johnlasseter, johnlasseter]","[jealousi, toy, boy, friendship, friend, rivalri, boynextdoor, newtoy, toycomestolif]"
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst, bradleypierce]","[joejohnston, joejohnston]","[boardgam, disappear, , newhom, reclus, giantinsect]"
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret, sophialoren]","[howarddeutch, howarddeutch]","[fish, bestfriend, duringcreditssting]"
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine, lelarochon]","[forestwhitaker, forestwhitaker]","[basedonnovel, interracialrelationship, singlemoth, divorc, chickflick]"
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort, kimberlywilliams-paisley]","[charlesshyer, charlesshyer]","[babi, midlifecrisi, confid, age, daughter, motherdaughterrelationship, pregnanc, contracept, gynecologist]"
...,...,...,...,...,...,...
40952,159550,The Last Brickmaker in America,[drama],"[sidneypoitier, wendycrewson, jayo.sanders, maryalice]","[greggchampion, greggchampion]",[friendship]
41172,392572,Rustom,"[thriller, romance]","[akshaykumar, , eshagupta, arjanbajwa]","[tinusureshdesai, tinusureshdesai]",[bollywood]
41225,402672,Mohenjo Daro,"[adventure, drama, history, romance]","[hrithikroshan, poojahegde, kabirbedi, arunodaysingh]","[ashutoshgowariker, ashutoshgowariker]",[bollywood]
41391,315011,Shin Godzilla,"[action, adventure, drama, horror, sciencefiction]","[hirokihasegawa, yutakatakenouchi, satomiishihara, kengokora]","[hideakianno, hideakianno]","[monster, godzilla, giantmonst, destruct, kaiju]"


In [ ]:
str_data['all_str_features'] = str_data['crew'] + str_data['cast'] + str_data['genres'] + str_data['keywords']
str_data['all_str_features'] = str_data['all_str_features'].apply(lambda x: ' '.join(x))
str_data

<ipython-input-13-bf3048e35900>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_data['all_str_features'] = str_data['crew'] + str_data['cast'] + str_data['genres'] + str_data['keywords']
<ipython-input-13-bf3048e35900>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  str_data['all_str_features'] = str_data['all_str_features'].apply(lambda x: ' '.join(x))


,id,title,genres,cast,crew,keywords,all_str_features
0,862,Toy Story,"[animation, comedy, family]","[tomhanks, timallen, donrickles, jimvarney]","[johnlasseter, johnlasseter]","[jealousi, toy, boy, friendship, friend, rivalri, boynextdoor, newtoy, toycomestolif]",johnlasseter johnlasseter tomhanks timallen donrickles jimvarney animation comedy family jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif
1,8844,Jumanji,"[adventure, fantasy, family]","[robinwilliams, jonathanhyde, kirstendunst, bradleypierce]","[joejohnston, joejohnston]","[boardgam, disappear, , newhom, reclus, giantinsect]",joejohnston joejohnston robinwilliams jonathanhyde kirstendunst bradleypierce adventure fantasy family boardgam disappear newhom reclus giantinsect
2,15602,Grumpier Old Men,"[romance, comedy]","[waltermatthau, jacklemmon, ann-margret, sophialoren]","[howarddeutch, howarddeutch]","[fish, bestfriend, duringcreditssting]",howarddeutch howarddeutch waltermatthau jacklemmon ann-margret sophialoren romance comedy fish bestfriend duringcreditssting
3,31357,Waiting to Exhale,"[comedy, drama, romance]","[whitneyhouston, angelabassett, lorettadevine, lelarochon]","[forestwhitaker, forestwhitaker]","[basedonnovel, interracialrelationship, singlemoth, divorc, chickflick]",forestwhitaker forestwhitaker whitneyhouston angelabassett lorettadevine lelarochon comedy drama romance basedonnovel interracialrelationship singlemoth divorc chickflick
4,11862,Father of the Bride Part II,[comedy],"[stevemartin, dianekeaton, martinshort, kimberlywilliams-paisley]","[charlesshyer, charlesshyer]","[babi, midlifecrisi, confid, age, daughter, motherdaughterrelationship, pregnanc, contracept, gynecologist]",charlesshyer charlesshyer stevemartin dianekeaton martinshort kimberlywilliams-paisley comedy babi midlifecrisi confid age daughter motherdaughterrelationship pregnanc contracept gynecologist
...,...,...,...,...,...,...,...
40952,159550,The Last Brickmaker in America,[drama],"[sidneypoitier, wendycrewson, jayo.sanders, maryalice]","[greggchampion, greggchampion]",[friendship],greggchampion greggchampion sidneypoitier wendycrewson jayo.sanders maryalice drama friendship
41172,392572,Rustom,"[thriller, romance]","[akshaykumar, , eshagupta, arjanbajwa]","[tinusureshdesai, tinusureshdesai]",[bollywood],tinusureshdesai tinusureshdesai akshaykumar eshagupta arjanbajwa thriller romance bollywood
41225,402672,Mohenjo Daro,"[adventure, drama, history, romance]","[hrithikroshan, poojahegde, kabirbedi, arunodaysingh]","[ashutoshgowariker, ashutoshgowariker]",[bollywood],ashutoshgowariker ashutoshgowariker hrithikroshan poojahegde kabirbedi arunodaysingh adventure drama history romance bollywood
41391,315011,Shin Godzilla,"[action, adventure, drama, horror, sciencefiction]","[hirokihasegawa, yutakatakenouchi, satomiishihara, kengokora]","[hideakianno, hideakianno]","[monster, godzilla, giantmonst, destruct, kaiju]",hideakianno hideakianno hirokihasegawa yutakatakenouchi satomiishihara kengokora action adventure drama horror sciencefiction monster godzilla giantmonst destruct kaiju


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
countvectorizer = CountVectorizer()
count_transform = countvectorizer.fit_transform(str_data['all_str_features'])
sim = cosine_similarity(count_transform)

In [ ]:
str_data = str_data.reset_index()
indexes = pd.Series(str_data.index, index=str_data['title'])
def content_based_str_recommand(movie_name,n):
    movie = indexes[movie_name]
    similar_movies = list(enumerate(sim[movie]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    similar_movies = similar_movies[0:n+1]
    similar_movies = [i[0] for i in similar_movies]
    return str_data.iloc[similar_movies]

In [ ]:
def content_base(movie_name):
  df = content_based_str_recommand(movie_name,10)
  df = df[1:11]
  titles = df['title']
  return titles


In [ ]:
test2 = content_base('Toy Story')
for i in test2:
  print(i)

Toy Story 2
Luxo Jr.
Cars 2
Toy Story of Terror!
Cars
Toy Story 3
A Bug's Life
Monsters, Inc.
Creature Comforts
The Lego Movie


In [ ]:
#collaborative_recommander

In [ ]:
ratings = ratings_small[['userId','movieId','rating']]
ratings

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [ ]:
movie_data2 = movies_metadata[['id','title']]
movie_data2 = movie_data2[movie_data2['id'].str.contains('-') == False]
movie_data2['id']=movie_data2['id'].astype(int)
movie_data2 = movie_data2[movie_data2['id'].isin(links_small2)]

In [ ]:
links = pd.merge(links,movie_data2,on='id')
links

,movieId,id,title
0,1,862,Toy Story
1,2,8844,Jumanji
2,3,15602,Grumpier Old Men
3,4,31357,Waiting to Exhale
4,5,11862,Father of the Bride Part II
...,...,...,...
9094,161944,159550,The Last Brickmaker in America
9095,162542,392572,Rustom
9096,162672,402672,Mohenjo Daro
9097,163056,315011,Shin Godzilla


In [ ]:
ratings = pd.merge(ratings,links,on='movieId',how='left')
ratings

,userId,movieId,rating,id,title
0,1,31,2.5,9909.0,Dangerous Minds
1,1,1029,3.0,11360.0,Dumbo
2,1,1061,3.0,819.0,Sleepers
3,1,1129,2.0,1103.0,Escape from New York
4,1,1172,4.0,11216.0,Cinema Paradiso
...,...,...,...,...,...
100039,671,6268,2.5,25461.0,Raising Victor Vargas
100040,671,6269,4.0,51927.0,Stevie
100041,671,6365,4.0,604.0,The Matrix Reloaded
100042,671,6385,2.5,1088.0,Whale Rider


In [ ]:
rate_data = ratings[['userId','rating','title']]
rate_data

,userId,rating,title
0,1,2.5,Dangerous Minds
1,1,3.0,Dumbo
2,1,3.0,Sleepers
3,1,2.0,Escape from New York
4,1,4.0,Cinema Paradiso
...,...,...,...
100039,671,2.5,Raising Victor Vargas
100040,671,4.0,Stevie
100041,671,4.0,The Matrix Reloaded
100042,671,2.5,Whale Rider


In [ ]:
rate_data = pd.pivot_table(rate_data,index='userId',columns='title',values='rating')
rate_data

title,$9.99,'Neath the Arizona Skies,"'night, Mother",(500) Days of Summer,...And God Created Woman,...And Justice for All,1-900,10,10 Attitudes,10 Cloverfield Lane,...,eXistenZ,loudQUIETloud: A Film About the Pixies,xXx,xXx: State of the Union,¡Three Amigos!,À Nous la Liberté,Æon Flux,İtirazım Var,Želary,’Round Midnight
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
movie_states = ratings.groupby('title').agg({'rating':[np.size,np.mean]})
movie_states.columns = ['rate_size','avg_rate']
popularmovies = movie_states['rate_size']>=70
popularmovies = movie_states[popularmovies]
popularmovies2 = popularmovies['avg_rate']>=3
popularmovies = popularmovies[popularmovies2]
popularmovies

,rate_size,avg_rate
title,,
12 Angry Men,78,4.294872
2001: A Space Odyssey,123,3.886179
A Beautiful Mind,114,3.951754
A Bug's Life,105,3.609524
A Christmas Story,75,4.046667
...,...,...
Who Framed Roger Rabbit,108,3.666667
Willy Wonka & the Chocolate Factory,148,3.753378
X-Men,127,3.559055


In [ ]:
def collaborative_recommand(movie_name,data,n):
    movie = data[movie_name]
    sim_movies = data.corrwith(movie)
    sim_movies = sim_movies.dropna()
    sim_movies =pd.DataFrame(sim_movies)
    sim_movies.columns = ['similarity']
    sim_movies = popularmovies.join(sim_movies)
    sim_movies =sim_movies.sort_values(by= ['similarity'],axis=0,ascending=False)
    sim_movies = sim_movies[1:n+1]
    return sim_movies

In [ ]:
def collaborative(movie_name):
  df = collaborative_recommand(movie_name,rate_data,10)
  df2 = list(df.index.values)
  return df2


In [ ]:
test3= collaborative('The Dark Knight')
for i in test3:
  print(i)

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


Crimson Tide
The Piano
Batman Begins
Superman
The Lion King
Maverick
Indiana Jones and the Last Crusade
Liar Liar
The Incredibles
The Bourne Supremacy


In [ ]:
#ensemble

In [ ]:
popularmovies = movie_states['rate_size']>=30
popularmovies = movie_states[popularmovies]
popularmovies2 = popularmovies['avg_rate']>=3.4
popularmovies = popularmovies[popularmovies2]
popularmovies

,rate_size,avg_rate
title,,
(500) Days of Summer,45,3.755556
10 Things I Hate About You,57,3.473684
12 Angry Men,78,4.294872
"20,000 Leagues Under the Sea",38,3.657895
2001: A Space Odyssey,123,3.886179
...,...,...
Wonder Boys,35,3.842857
X-Men,127,3.559055
X2,81,3.617284


In [ ]:
def ensemble(movie_name):
  str_data2 = content_based_str_recommand(movie_name,40)
  df =  rate_data[str_data2['title']]
  similarmovies = collaborative_recommand(movie_name,df,8)
  df2 = list(similarmovies.index.values)
  return df2

In [ ]:
test4= ensemble('The Dark Knight')
for i in test4:
  print(i)

Batman Begins
Superman
The Dark Knight Rises
Batman
The Prestige
Interstellar
Inception
(500) Days of Summer


/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [ ]:
test5 = ensemble('Toy Story')
for i in test5:
  print(i)

Toy Story 3
Toy Story 2
A Bug's Life
Monsters, Inc.
Big
Pinocchio
Chicken Run
That Thing You Do!
